In [5]:
import praw
import pandas as pd
import json
from datetime import datetime
from confluent_kafka import Producer, Consumer

In [6]:
# Kafka configuration
bootstrap_servers = "localhost:9092"
kafka_topic = "reddit"
producer = Producer({'bootstrap.servers': bootstrap_servers})

In [ ]:
# Configuration du consommateur
config = {'bootstrap.servers': bootstrap_servers, 'group.id': 'Xmen', 'auto.offset.reset': 'earliest'}

# Création du consommateur
consumer = Consumer(config)

# S'abonner à un ou plusieurs topics
topics = ['reddit']
consumer.subscribe(topics)

tweets_consumer = []
# Consommer les messages
while True:
    
    message = consumer.poll(1.0)  # Attendre pendant 1 seconde au maximum
    
    if message is None:
        continue
    
    if message.error():
        print("Erreur lors de la consommation : {}".format(message.error()))
        continue
    
    # Traitement du message
    # tweets_consumer.append(eval(message.value().decode('latin1', 'surrogateescape')))
    print("Message reçu : {}".format(message.value().decode('latin1')))
    
    df_new_tweets = pd.DataFrame([eval(message.value().decode('latin1'))])
    try:
        df_tweets = pd.read_csv("df_tweets.csv")
    except:
        try:
            df_new_tweets.to_csv("df_tweets.csv", index=False)
            continue
        except:
            continue
    if not df_new_tweets.equals(df_tweets):
        try:
            concat_data = pd.concat([df_tweets, df_new_tweets])
            concat_data = concat_data.drop_duplicates(keep="last")
            concat_data.to_csv("df_tweets.csv", index=False)
        except:
            continue
        
# Fermeture du consommateur
consumer.close()

%4|1684524895.380|MAXPOLL|rdkafka#consumer-5| [thrd:main]: Application maximum poll interval (300000ms) exceeded by 186ms (adjust max.poll.interval.ms for long-running message processing): leaving group
